In [14]:
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from keras import models, Input
from keras import optimizers as opt
from keras import backend as K
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [15]:
from dataset import load_dataset, load_labels, split_data, format_labels
from features import time_series_features, fractal_features, entropy_features, hjorth_features, freq_band_features
import variables as v

# Variables

In [47]:
data_type = "ica_filtered"
test_type = "Arithmetic"

# Load Dataset

In [75]:
dataset_ = load_dataset(data_type=data_type, test_type=test_type)
dataset = split_data(dataset_, v.SFREQ)

In [94]:
#label_ = load_labels()
label = [0,1,2]
label = format_labels(label_, test_type=test_type, epochs=dataset.shape[1])
print(label)

[ True  True  True ... False False False]


# Compute Features

In [86]:
 features = time_series_features(dataset)
# freq_bands = np.array([1, 4, 8, 12, 30, 50])
# features = freq_band_features(dataset, freq_bands)
# features = hjorth_features(dataset)
# features = entropy_features(dataset)
#features = fractal_features(dataset)

In [87]:
data = features

# k-NN Classifier

In [91]:
x, x_test, y, y_test = train_test_split(
    data, label, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.25, random_state=1)
scaler = MinMaxScaler()
scaler.fit(x_train)
x = scaler.transform(x)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

param_grid = {
    'leaf_size': range(50),
    'n_neighbors': range(1, 10),
    'p': [1, 2]
}
split_index = [-1 if x in range(len(x_train)) else 0 for x in range(len(x))]
ps = PredefinedSplit(test_fold=split_index)
knn_clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=ps, refit=True)
knn_clf.fit(x, y)

C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
18 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py", line 666, in _validate_params
    valida

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': range(0, 50), 'n_neighbors': range(1, 10),
                         'p': [1, 2]})

In [92]:
y_pred = knn_clf.predict(x_test)
y_true = y_test

In [93]:
print(metrics.classification_report(y_true, y_pred, target_names=['Stress', 'Goodmod', 'Calm']))
print(metrics.confusion_matrix(y_true, y_pred))

ValueError: Number of classes, 2, does not match size of target_names, 3. Try specifying the labels parameter